In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as pl
from scipy.stats import linregress

In [ ]:
temp = np.load('../data/temp_c1.npy')
co2_emissions = np.load('../data/co2_emissions_c1.npy')
co2_cumulative = np.load('../data/co2_cumulative_c1.npy')

In [ ]:
netzero_times = np.array([co2_emissions<0]).argmax(axis=1).squeeze()
netzero_times

In [ ]:
co2_cumulative[netzero_times, :]

In [ ]:
netzero_to_2100_cumulative_emissions = np.zeros(97)
for iscen in tqdm(range(97)):
    netzero_to_2100_cumulative_emissions[iscen] = (co2_cumulative[-1,iscen] - co2_cumulative[netzero_times[iscen], iscen])

In [ ]:
# exclude scenarios that don't reach net zero
netzero_to_2100_cumulative_emissions[netzero_to_2100_cumulative_emissions > 0] = np.nan
netzero_to_2100_cumulative_emissions

In [ ]:
peak_times = np.zeros((97, 1001))
peak_times = temp.argmax(axis=0)

In [ ]:
temperature_drawdown = np.zeros((97,1001))
for iscen in tqdm(range(97)):
    for iconfig in range(1001):
        temperature_drawdown[iscen, iconfig] = temp[netzero_times[iscen],iscen,iconfig]-temp[-1,iscen,iconfig]

In [ ]:
pl.hist(temperature_drawdown[2,:])

In [ ]:
pc05 = np.percentile(temperature_drawdown, 5, axis=1)
pc50 = np.percentile(temperature_drawdown, 50, axis=1)
pc95 = np.percentile(temperature_drawdown, 95, axis=1)

netzero = ~np.isnan(netzero_to_2100_cumulative_emissions)

In [ ]:
pl.scatter(-pc50[netzero], netzero_to_2100_cumulative_emissions[netzero])
pl.scatter(-pc05[netzero], netzero_to_2100_cumulative_emissions[netzero])
pl.scatter(-pc95[netzero], netzero_to_2100_cumulative_emissions[netzero])
pl.xlabel('Temperature peak to 2100')
pl.ylabel('Cumulative CO2, net zero to 2100')

sl50, ic50, _, _, _ = linregress(-pc50[netzero], netzero_to_2100_cumulative_emissions[netzero])
x = np.linspace(-0.35, 0)
pl.plot(x, sl50*x + ic50)
print(sl50, ic50)

sl05, ic05, _, _, _ = linregress(-pc05[netzero], netzero_to_2100_cumulative_emissions[netzero])
x = np.linspace(-0.13, 0)
pl.plot(x, sl05*x + ic05)
print(sl05, ic05)

sl95, ic95, _, _, _ = linregress(-pc95[netzero], netzero_to_2100_cumulative_emissions[netzero])
x = np.linspace(-0.5, 0)
pl.plot(x, sl95*x + ic95)
print(sl95, ic95)

In [ ]:
3664/sl50  # effective TCRE

In [ ]:
ic50/sl50  # -ZEC

In [ ]:
# I don't think ensemble averaging approach is working in the all-forcings case
3664/sl05, 3664/sl95

In [ ]:
ic05/sl05, ic95/sl95  # take negative of

In [ ]:
peak_times_median_warming = np.zeros((97))
peak_times_median_warming = np.median(temp, axis=2).argmax(axis=0)
peak_times_median_warming

In [ ]:
peak_times_median_warming.shape

In [ ]:
co2_emissions.shape#[[peak_times_median_warming]]

In [ ]:
x = np.ones(97) * np.nan

for iscen in range(97):
    if peak_times_median_warming[iscen] < 106:
        x[iscen] = co2_emissions[peak_times_median_warming[iscen], iscen]

In [ ]:
pl.hist(x)

In [ ]:
np.nanmedian(x)

In [ ]:
np.median(temp, axis=2)[:, 0]

In [ ]:
peak_times_median_warming

In [ ]:
co2_emissions[:, 0]

In [ ]:
peak_times_median_warming

In [ ]:
np.median(np.median(temp, axis=2), axis=1)

In [ ]:
fair_params_df = pd.read_csv('../data/calibrated_constrained_parameters.csv', index_col=0)

In [ ]:
dataout = np.zeros((1001,5))

for iconfig, config in enumerate(fair_params_df.index):
    sl, ic, _, _, _ = linregress(-temperature_drawdown[netzero,iconfig], netzero_to_2100_cumulative_emissions[netzero])
    print(sl, ic, 3664/sl, -ic/sl)
    dataout[iconfig, :] = [config, sl, ic, 3664/sl, -ic/sl]

In [ ]:
df = pd.DataFrame(dataout, columns=['config', 'slope', 'intercept', 'tcre_postpeak', 'zec'])

In [ ]:
df = df.astype({'config': 'int32'})

In [ ]:
df.set_index('config', inplace=True)

In [ ]:
df

In [ ]:
df.to_csv('../data/fair_1001_c1_scenarios_effective_tcre_zec.csv')

In [ ]:
temp.shape

In [ ]:
co2_cumulative.shape

In [ ]:
pl.plot(co2_cumulative[netzero_times[1]+5:, 1], temp[netzero_times[1]+5:, 1, :]);